In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv(r'E:\MSBA\Courses\Spring\Text Analytics\Project\Dataset\Final\br_model_phx_2014-2018.csv')

In [3]:
review =df['text_review']
name = df['name']

In [4]:
df_2 = pd.DataFrame(data={'title':name,'review':review})

In [5]:
df_2.drop_duplicates(keep='first',inplace = True)

In [6]:
df_2['index']=range(df_2.shape[0])
df_2.set_index('index',inplace=True)

In [7]:
review=df_2['review'].tolist()
len(review)

5929

In [8]:
tokens=[]
for i in range(len(review)):
    token = nltk.word_tokenize(review[i])
    token = [w.lower() for w in token]
    token = [w for w in token if w.isalpha()]
    tokens.append(token)


In [9]:
len(tokens)

5929

In [10]:
stopwords = stopwords.words('english')

In [11]:
nostopwords=[]
for i in range(len(tokens)):
    words = tokens[i]
    words_nostopwords = [w for w in words if w not in stopwords]
    nostopwords.append(words_nostopwords)

In [12]:
len(nostopwords)

5929

In [13]:
stem = []
porter = nltk.PorterStemmer()
for i in range(len(nostopwords)):
    words = nostopwords[i]
    stem1 = [porter.stem(w) for w in words]
    stem.append(stem1)


In [14]:
len(stem)

5929

In [15]:
stem = pd.Series(stem)

In [16]:
df_2['review_words']=stem


In [17]:
df_2

,title,review,review_words
index,,,
0,Chino Bandido,A must go destination in Phoenix.\nChinese - M...,"[must, go, destin, phoenix, chines, mexican, j..."
1,Joe's Midnight Run,THE late night spot in Phx for high quality Am...,"[late, night, spot, phx, high, qualiti, americ..."
2,32 Shea,"The restaurant has been sold, so I will reserv...","[restaur, sold, reserv, judgement, month, see,..."
3,Pita Bistro,I think I'm addicted to their homemade pita ch...,"[think, addict, homemad, pita, chip, hummu, de..."
4,Harumi Sushi,This is just your average sushi place The Purp...,"[averag, sushi, place, purpl, rice, pretti, co..."
...,...,...,...
5924,Arroy Thai,"Smaller Servings, OK Food, Higher Prices \n\nA...","[smaller, serv, ok, food, higher, price, tri, ..."
5925,Fired Pie,Went there because the TV advertised National ...,"[went, tv, advertis, nation, pizza, day, natio..."
5926,Church's Chicken,I haven't ordered chicken from this location i...,"[order, chicken, locat, rememb, first, ask, sp..."


In [18]:
df_3 =df_2.groupby(by='title')['review_words'].sum()

In [19]:
df_3=pd.DataFrame(df_3)

In [20]:
for i in range(len(df_3['review_words'])):
    df_3['review_words'][i]=','.join(df_3['review_words'][i])

In [21]:
count = CountVectorizer()
count_matrix = count.fit_transform(df_3['review_words'])

In [22]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.08746385, 0.07888007, ..., 0.10516693, 0.52821066,
        0.12681706],
       [0.08746385, 1.        , 0.19087136, ..., 0.03331894, 0.27904066,
        0.2236817 ],
       [0.07888007, 0.19087136, 1.        , ..., 0.09929231, 0.21925814,
        0.22616243],
       ...,
       [0.10516693, 0.03331894, 0.09929231, ..., 1.        , 0.12304447,
        0.11809198],
       [0.52821066, 0.27904066, 0.21925814, ..., 0.12304447, 1.        ,
        0.29922899],
       [0.12681706, 0.2236817 , 0.22616243, ..., 0.11809198, 0.29922899,
        1.        ]])

In [31]:
indices = pd.Series(df_3.index)
recommended=[]

def recommendations(title, cosine_sim = cosine_sim):
    
    
    idx = indices[indices == title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    top_5_indexes = list(score_series.iloc[1:6].index)
          
    for i in top_5_indexes:
        recommended.append(list(df_3.index)[i])
        
    return recommended

In [32]:
recommendations('Hooters')

Index(['Buffalo Wild Wings', 'Gallagher's', 'Native Grill & Wings',
       'The Attic Ale House', 'Red Robin Gourmet Burgers'],
      dtype='object', name='title')

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_3['review_words'])

In [62]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.00000000e+00, 6.68958537e-03, 7.95858422e-03, ...,
        4.87682191e-03, 9.87394260e-02, 7.74785548e-03],
       [6.68958537e-03, 1.00000000e+00, 6.04931783e-03, ...,
        2.34651942e-04, 1.58669419e-02, 1.16642533e-02],
       [7.95858422e-03, 6.04931783e-03, 1.00000000e+00, ...,
        3.21114785e-03, 1.14934271e-02, 1.13050933e-02],
       ...,
       [4.87682191e-03, 2.34651942e-04, 3.21114785e-03, ...,
        1.00000000e+00, 6.10845412e-03, 3.26356079e-03],
       [9.87394260e-02, 1.58669419e-02, 1.14934271e-02, ...,
        6.10845412e-03, 1.00000000e+00, 1.97290330e-02],
       [7.74785548e-03, 1.16642533e-02, 1.13050933e-02, ...,
        3.26356079e-03, 1.97290330e-02, 1.00000000e+00]])

In [63]:
indices = pd.Series(df_3.index)
recommended=[]

def recommendations(title, cosine_sim = cosine_sim):
    
    
    idx = indices[indices == title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    top_5_indexes = list(score_series.iloc[1:6].index)
    for i in top_5_indexes:
        recommended.append(list(df_3.index)[i])
        
    return recommended

In [64]:
recommendations('Hooters')

['The Attic Ale House',
 'Mellow Mushroom',
 'Buffalo Wild Wings',
 'Red Robin Gourmet Burgers',
 'Wingstop']